## Enron Email Data Transformation and Analysis

### Import the required dependencies

In [1]:
import re
import pandas as pd
import os
import numpy as np
from datetime import datetime
from email.utils import parsedate_to_datetime
# below is only needed if visualization is needed
import math
import networkx as nx
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import percentileofscore

### Load the structured data from the profiling stage

In [2]:
# Define the path to the input and output directories and files
INPUT = 'input'
MAILDIR = 'maildir'
OUTPUT = 'output'
STRUCTURED_DATA = 'structured_data_'
PROFILING_OUTPUT = 'profiling_output_'
TRANSFORMATION1_OUTPUT = 'analysis1_transformation_and_enriching_output_'
TRANSFORMATION2_OUTPUT = 'analysis2_transformation_and_enriching_output_'
ANALYSIS1_VIS_OUTPUT = 'analysis_1_visualisation_output_'
ANALYSIS2_VIS_OUTPUT = 'analysis_2_visualisation_output_'
IMPORT_SUCCESS = 'SUCCESS'

input_dir = os.path.join(
    os.path.abspath('..'),
    INPUT,
    MAILDIR
)
output_dir = os.path.join(
    os.path.abspath('..'),
    OUTPUT
)
structured_data_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    STRUCTURED_DATA + datetime.now().strftime("%Y%m%d_%H%M%S")
)
profiling_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    PROFILING_OUTPUT + datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'
)
transformation1_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    TRANSFORMATION1_OUTPUT + datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'
)
transformation2_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    TRANSFORMATION2_OUTPUT + datetime.now().strftime("%Y%m%d_%H%M%S") + '.csv'
)
analysis_question_1_visualisation_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    ANALYSIS1_VIS_OUTPUT + datetime.now().strftime("%Y%m%d_%H%M%S") + '.html'
)
analysis_question_2_visualisation_output_path = os.path.join(
    os.path.abspath('..'),
    OUTPUT,
    ANALYSIS2_VIS_OUTPUT + datetime.now().strftime("%Y%m%d_%H%M%S") + '.html'
)

In [3]:
# this block is needed if we'd like to read the df from saved
# structured email data files in csv format
# Please note: the target location is in the output folder in the project !
STRUCTURED_DATA_FN_PATTERN = '^structured_data_' # provide the saved structured data csv file name regex pattern
list_of_df = []
for root, dirs, files in os.walk(output_dir):
    if files:
        for file in files: # iterate through all files matching the pattern
            if re.match(STRUCTURED_DATA_FN_PATTERN, file) is not None:
                this_df = pd.read_csv(os.path.join(root,file), index_col=None, header=0)
                list_of_df.append(this_df) # and append each one into the whole list for combination
df = pd.concat(list_of_df, axis=0, ignore_index=True) # combine and form the full email data

### Cleanse the data and apply curations

In [4]:
def _clean_and_sort_comma_separated_email_addresses(s):
    """
    This is a helper function to tidy up comma delimited strings
    by removing all whitespace, newlines and carriage returns
    and then sorting the strings alphabetically

    parameters:
        s: the string to tidy up
    returns:
        the tidied up string
    """
    if type(s) == str:
        return ','.join(sorted([re.sub(r'[\r\n\s]+','',i) for i in s.split(',')]))
    else:
        return s

In [5]:
# convert the comma delimited email strings in the From, To, Cc and Bcc fields into lists
# so they can be cleansed iteratively. After the treatment, convert them back to comma-delimited strings
# so that they can be properly de-duplicated.
df['From'] = df['From'].apply(lambda x: _clean_and_sort_comma_separated_email_addresses(x))
df['To'] = df['To'].apply(lambda x: _clean_and_sort_comma_separated_email_addresses(x))
df['Cc'] = df['Cc'].apply(lambda x: _clean_and_sort_comma_separated_email_addresses(x))
df['Bcc'] = df['Bcc'].apply(lambda x: _clean_and_sort_comma_separated_email_addresses(x))
# Also parse the Date values into UTC datetime stamps so they are comparable and can be properly deduplicated
df['Date'] = df['Date'].apply(lambda x: parsedate_to_datetime(x) if type(x) == str else x)

In [6]:
# Curation 1 Extract the relevant fields from the df
df_extracted = df[['Message-ID', 'From', 'To', 'Cc', 'Bcc', 'Date', 'Subject']].copy()
# Curation 2 Unify the data to lower cases so they can be properly deduplicated
for column in df_extracted.columns:
    # this is only needed for from, to, cc and bcc columns
    # we don't make the Subject all lower case because the upper/lower cases
    # carry actual information, and we don't want to lose that
    if column in ['From', 'To', 'Cc', 'Bcc']:
        df_extracted[column] = df_extracted[column].str.lower() 
# Curation 3 Convert the Date column from datetime.datetime to np.datetime64 so it can be properly deduplicated
df_extracted['Date'] = pd.to_datetime(df_extracted['Date'], utc=True)
# Curation 4 Remove invalid dates
# Invalid dates are defined as all dates before 1998-01-01 and after 2002-12-31 (exclusive)
df_extracted = df_extracted.loc[
    ((df_extracted['Date'].dt.year*100 + df_extracted['Date'].dt.month) >= 199801) & 
    ((df_extracted['Date'].dt.year*100 + df_extracted['Date'].dt.month) <= 200212)
]
# Curation 5 Remove the duplicates (the same email can be saved as multiple copies in different people's mailboxes)
df_extracted = df_extracted.groupby(['From', 'To', 'Cc', 'Bcc', 'Date', 'Subject'], dropna=False)['Message-ID'].last().reset_index()


In [7]:
# Curation 6 We are only interested in genuine human email addresses
# so the valid email addresses are defined as matching the following regex
# ^[a-z0-9._%+-]+@[a-z0-9.-]+\.[a-z]{2,}$
# By doing this we can filter out system email addresses
# From column
email_regex_pattern = r"^[a-z0-9.'_%+-<]+@[a-z0-9.-]+\.[a-z]{2,}>*?$"
df_extracted.loc[~df_extracted['From'].str.match(email_regex_pattern),['From']] = np.nan
# To, Cc and Bcc columns:
# convert string to lists first
df_extracted['To'] = df_extracted['To'].apply(lambda x: x.split(',') if type(x) == str else x)
df_extracted['Cc'] = df_extracted['Cc'].apply(lambda x: x.split(',') if type(x) == str else x)
df_extracted['Bcc'] = df_extracted['Bcc'].apply(lambda x: x.split(',') if type(x) == str else x)
# Then iterate through each list and keep only the valid email addresses
email_regex = re.compile(email_regex_pattern)
df_extracted['To'] = df_extracted['To'].apply(lambda x: [i for i in x if email_regex.match(i)] if isinstance(x, list) else x)
df_extracted['Cc'] = df_extracted['Cc'].apply(lambda x: [i for i in x if email_regex.match(i)] if isinstance(x, list) else x)
df_extracted['Bcc'] = df_extracted['Bcc'].apply(lambda x: [i for i in x if email_regex.match(i)] if isinstance(x, list) else x)
# If a list is reduced to zero after the cleansing, change the value to np.nan
df_extracted['To'] = df_extracted['To'].apply(lambda x: np.nan if (isinstance(x, list) and not x) else x)
df_extracted['Cc'] = df_extracted['Cc'].apply(lambda x: np.nan if (isinstance(x, list) and not x) else x)
df_extracted['Bcc'] = df_extracted['Bcc'].apply(lambda x: np.nan if (isinstance(x, list) and not x) else x)


In [8]:
# Curation 7 Remove records with empty From, empty To+Cc+Bcc and empty Date fields, because they are not useful for our analysis
df_extracted = df_extracted[
    ~(
        df_extracted['From'].isna() | 
        (df_extracted['To'].isna() & df_extracted['Cc'].isna() & df_extracted['Bcc'].isna()) | 
        df_extracted['Date'].isna()
    )
].reset_index(drop=True)


### Transform and normalise the data into separate tables
We need the following 3 tables to perform the analysis:
- email_from: a table containing the relationship between Message-ID and From
- email_to: a table containing the relationship between Message-ID, and the email address in To, Cc and Bcc, and the type of messaging (to, cc or bcc)
- email_datetime: a table containing the relationship between Message-ID and Date

In [9]:
# get the email_from table.
df_email_from = df_extracted[['Message-ID', 'From']].copy()
# get the email_table
df_email_to = df_extracted[['Message-ID', 'To', 'Cc', 'Bcc']].copy()
# get the email_datetime table
df_email_datetime = df_extracted[['Message-ID', 'Date']].copy()

In [10]:
# for the email_to table we need to normalise it so that each email_address contains only one value
df_email_to = df_email_to.melt(id_vars=['Message-ID'], value_vars=['To', 'Cc', 'Bcc'])
df_email_to = df_email_to.rename(columns={'variable': 'message_type', 'value': 'message_to'})
df_email_to = df_email_to.explode('message_to').reset_index(drop=True)
df_email_to.dropna(inplace = True, ignore_index= True)

In [11]:
# for analysis question 1 : the change of the external communication frequencies before and after 
# the Enron scandal, we need to join the email_from and email_to tables to get the information of
# whether the email is internal or external. 
# In the enriching step, we then need to join the product table to the email_datetime
# table, and aggregate the total number of emails grouped by both month and internal/external.
df_analysis_1 = df_email_from.merge(df_email_to, on='Message-ID', how='inner')
df_analysis_1 = df_analysis_1.merge(df_email_datetime, on='Message-ID', how='inner')

In [12]:
# for analysis question 2 : the Enron internal network, we need to join the email_from and email_to 
# tables to get the information of sender-receiver pairs. We then need to order the sender and receiver
# pairs alphabetically so that the pairs are no longer vectorized.
# In the enriching step, we then need to aggregate the number of total emails grouped by the pairs.
# The number of total emails can be a presentation of the strength of the relationship between the pairs.
df_analysis_2 = df_email_from.merge(df_email_to, on='Message-ID', how='inner')
# remove the email pairs with the same sender and receiver
df_analysis_2 = df_analysis_2[df_analysis_2['From'] != df_analysis_2['message_to']].copy().reset_index(drop=True)
df_analysis_2['email_A'] = np.minimum(df_analysis_2['From'], df_analysis_2['message_to']) 
df_analysis_2['email_B'] = np.maximum(df_analysis_2['From'], df_analysis_2['message_to']) 


### Enrich the transformed data for the analysis questions

For analysis question 1, we need to create the following columns:
1. communication_type: internal or external
2. communication_month: a higher level of time granularity (year-month)

please note that one email can have both internal and external from-and-to address pairs,
and we treat all email address pairs in the same email as separate communication instances
The results are visualised in the bar chart below

In [13]:
# calculate if a pair of communication emails is of the relationship internal or external by their domains
internal_email_pattern = r"^[a-z0-9.'_%+-<]+@enron\.com[>]?$"
df_analysis_1['communication_type'] = 'external'
df_analysis_1.loc[
    df_analysis_1['From'].str.contains(internal_email_pattern) &
    df_analysis_1['message_to'].str.contains(internal_email_pattern),
    'communication_type'
] = 'internal'
df_analysis_1['communication_month'] = df_analysis_1['Date'].dt.strftime('%Y-%m')
# Aggregate the result to the year-month level
df_analysis_1_result = df_analysis_1.groupby(['communication_type', 'communication_month'], dropna=False).size().reset_index().rename(columns={0:'instance_count'})
df_analysis_1_result.to_csv(transformation1_output_path, index=False)

In [14]:
# Create the line chart
fig = px.line(df_analysis_1_result, x='communication_month', y='instance_count', color='communication_type', title='Enron Monthly Email Communication Intances by Internal/External')

# Save the output for further analysis (see the attached summary report)
fig.write_html(analysis_question_1_visualisation_output_path)

For analysis question 2, we need to create the following caculated column:
1. communication_weight

We'd like to define the communication_weight of each instance per below:
- for instances sending directly To an email address, the weight is 1
- for instances sending as Cc to an email address, the weight is 0.2
- for instances sending as Bcc to an email address, the weight is 0.5

similar to analysis question 1, multiple pairs in the same email are considered to contribute
to a pair of email addresses as separate communication instance the results are visualised in a network graph

In [15]:
# Calculate the weight of communication by its type (To, Cc or Bcc)
df_analysis_2['communication_weight'] = 1
df_analysis_2.loc[df_analysis_2['message_type'] == 'Cc', 'communication_weight'] = 0.2
df_analysis_2.loc[df_analysis_2['message_type'] == 'Bcc', 'communication_weight'] = 0.5
# sum up the weight and group by the email pairs
df_analysis_2_result = df_analysis_2.groupby(['email_A', 'email_B'], dropna=False)['communication_weight'].sum().reset_index()
df_analysis_2_result.to_csv(transformation2_output_path, index=False)

Because we need to keep the number of nodes viable for a network graph 
so we are only interested in Enron internal significant connections which
have the communciation weights over 100

In [16]:
# Select the effective internal email pairs
internal_email_pattern = r"^[a-z0-9.'_%+-<]+@enron\.com[>]?$"
df_analysis_2_result_effective = df_analysis_2_result[
    df_analysis_2_result['email_A'].str.contains(internal_email_pattern) &
    df_analysis_2_result['email_B'].str.contains(internal_email_pattern) &
    (df_analysis_2_result['communication_weight'] >= 100.0)
].copy().reset_index(drop=True)

# Instantiate the network from the selected email pairs and communication weights
G = nx.from_pandas_edgelist(df_analysis_2_result_effective, 'email_A', 'email_B', ['communication_weight'])
pos = nx.spring_layout(G, k=2/math.sqrt(len(G.nodes())))  # you can use other layout algorithms as well
nx.set_node_attributes(G, pos, 'pos')

# Calculate sizes based on total communication_weight
sizes = {}
for node in G.nodes():
    sizes[node] = sum(weight for _, _, weight in G.edges(node, data='communication_weight'))
nx.set_node_attributes(G, sizes, 'size')

In [17]:
# Plot the gragh
edge_x = []
edge_y = []
edge_weights = []
edge_note = []
edge_note_points_x = []
edge_note_points_y = []

for edge in G.edges(data='communication_weight'):
    x0, y0 = G.nodes[edge[0]]['pos'] # extract the locations of node_0 (email) from edges (communication)
    x1, y1 = G.nodes[edge[1]]['pos'] # extract the locations of node_1 (email) from edges (communication)
    edge_note.append(f"{edge[0]} - {edge[1]}: {edge[2]}") # build the edge note list
    edge_x.append([x0, x1, None])
    edge_y.append([y0, y1, None])
    edge_note_points_x.append((x0 + x1)/2) # assuming values positive/get midpoint
    edge_note_points_y.append((y0 + y1)/2) # assumes positive vals/get midpoint
    edge_weights.append(edge[2])
edge_weights_colour_scale = []
# use each edge's weight to scale it to a degree of grey
for w in edge_weights:
    scaled_weight = (1 - percentileofscore(edge_weights, w, 'rank') / 100) * 255
    edge_weights_colour_scale.append(f"rgb({scaled_weight},{scaled_weight},{scaled_weight})")
# Also calculate the minimum and maximum of the degree of grey to be used in the chart color bar
colorscale_min = (1 - percentileofscore(edge_weights, min(edge_weights), 'rank') / 100) * 255
colorscale_max = (1 - percentileofscore(edge_weights, max(edge_weights), 'rank') / 100) * 255

# Build the edge lines, each with a different colour depending on its weight
edge_traces = []
for _x, _y, _w in zip(edge_x, edge_y, edge_weights_colour_scale):
    edge_trace = go.Scatter(
        x=_x,
        y=_y,
        line=dict(color=_w, width=1),
        hoverinfo='none',
        mode='lines',
        showlegend=False
    )
    edge_traces.append(edge_trace)

# Build the edge hovering points with notes at the medium location of the two nodes
edge_note_trace = go.Scatter(
    x=edge_note_points_x,
    y=edge_note_points_y,
    mode='markers',
    marker=dict(
        opacity=0, 
        color=edge_weights,
        colorbar=dict(
            title="Communication<BR>Weight"
        ),
        colorscale=[[0, f"rgb({colorscale_min},{colorscale_min},{colorscale_min})"], [1, f"rgb({colorscale_max},{colorscale_max},{colorscale_max})"]]
    ),
    showlegend=False,
    hoverinfo='text',
    text=edge_note
)

# Retrieve the nodes locations from the network and plot them in a scatter chart
node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=[f"{node} : {round(G.nodes[node]['size'],2)}" for node in G.nodes()],
    marker=dict(
        size=[round(math.sqrt(G.nodes[node]['size'])/2.5,2) for node in G.nodes()]
    )
)

# Plot all the graph from above on the same planner
fig = go.Figure(
    data=[edge_note_trace, node_trace, *edge_traces],
    layout=go.Layout(
        showlegend=False,
        hovermode='closest',
        title="Enron Major Connection Network",
        margin=dict(l=20, r=20, t=25, b=15),
        xaxis=dict(
            title=None,  # Removes x-axis title
            showgrid=False,  # Removes x-axis grid lines
            showticklabels=False,  # Removes x-axis tick labels
            ticks=""  # Removes x-axis tick marks
        ),
        yaxis=dict(
            title=None,  # Removes x-axis title
            showgrid=False,  # Removes x-axis grid lines
            showticklabels=False,  # Removes x-axis tick labels
            ticks=""  # Removes x-axis tick marks
        )
    )
)

# Save the output for further analysis (see the attached summary report)
fig.write_html(analysis_question_2_visualisation_output_path)